In [ ]:
import pandas as pd

df_18 = pd.read_csv('../MIGTABO19X.csv',sep="|")
df_columns_18 = pd.read_csv('Campos MIGTAB019.csv',sep=";")
#df_18.drop('Unnamed: 3',axis=1,inplace= True)
df_18.columns = df_columns_18['NombreColumna'].tolist()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

C:\Users\Fabian - PC\AppData\Local\Temp\ipykernel_18900\3853580854.py:3: DtypeWarning: Columns (2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv('../MIGTABO19X.csv',sep="|")


In [13]:
df_18.to_csv('../MIGTABO19XCampos.csv')

In [24]:
df_18.head(5)

,ro_operacion,ro_concepto,ro_signo,ro_factor,ro_referencial,ro_signo_reajuste,ro_factor_reajuste,ro_referencial_reajuste,ro_valor,ro_porcentaje_efa,ro_garantia,ro_financiado
0,5001,INT,NaN,0.0,T_FIJA,NaN,NaN,NaN,0.0,29.3334,0,NaN
1,5001,IMO,NaN,0.0,T_USURA,NaN,0.0,NaN,0.0,0.0000,0,NaN
2,5002,CAP,NaN,0.0,NaN,NaN,0.0,NaN,3672326.0,0.0000,0,NaN
3,5002,INT,NaN,0.0,T_FIJA,NaN,NaN,NaN,0.0,24.9962,0,NaN
4,5002,IMO,NaN,0.0,T_USURA,NaN,0.0,NaN,0.0,0.0000,0,NaN


#### Validación Campos obligatorios que no vengan nulos

In [ ]:
df_columnasObligatorias = df_columns_18[df_columns_18['Obligatorio'] == 'S']['NombreColumna'].tolist()
df_Final = df_18[df_columnasObligatorias]
#df_Final.shape
nulos = df_Final.isnull().sum().reset_index()
nulos.columns = ['NombreColumna','CantidadNulos']
df_Columnas = nulos[nulos['CantidadNulos']> 0 ]['NombreColumna'].head(4).tolist()


[]

In [14]:
errores = {}

In [20]:
df_18["ro_operacion"] = pd.to_numeric(df_18["ro_operacion"], errors="coerce")
errores["ro_operacion"] = df_18[df_18["ro_operacion"].isnull() | (df_18["ro_operacion"] == 0)]


In [22]:
errores["ro_concepto"] = df_18[df_18["ro_concepto"].isnull() | (df_18["ro_concepto"].astype(str).str.strip() == "")]


In [25]:
# Validación 3: ro_signo
def validar_signo(row):
    signo = str(row["ro_signo"]).strip()
    concepto = row["ro_concepto_normalizado"]
    if concepto in ["INTERES", "INTERES MORA"]:
        return signo not in ["+", "-"]
    elif concepto == "CAPITAL":
        return pd.notna(row["ro_signo"]) and signo != ""
    return False

errores["ro_signo"] = df_18[df_18.apply(validar_signo, axis=1)]

KeyError: 'ro_concepto_normalizado'

In [23]:
errores

{'ro_operacion': Empty DataFrame
 Columns: [ro_operacion, ro_concepto, ro_signo, ro_factor, ro_referencial, ro_signo_reajuste, ro_factor_reajuste, ro_referencial_reajuste, ro_valor, ro_porcentaje_efa, ro_garantia, ro_financiado]
 Index: [],
 'ro_concepto': Empty DataFrame
 Columns: [ro_operacion, ro_concepto, ro_signo, ro_factor, ro_referencial, ro_signo_reajuste, ro_factor_reajuste, ro_referencial_reajuste, ro_valor, ro_porcentaje_efa, ro_garantia, ro_financiado]
 Index: []}